In [3]:
import rasterio
import numpy as np
from tkinter import Tk, filedialog, simpledialog, messagebox
from mapclassify import NaturalBreaks
import os

# Helper: Select two .tiff files
def select_tiff_files():
    root = Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(
        title="Select Two .tiff Files", filetypes=[("TIFF files", "*.tif *.tiff")]
    )
    if len(file_paths) != 2:
        raise ValueError("You must select exactly two TIFF files.")
    return file_paths

# Get natural breaks classification bins
def get_class_bins(array, n_classes):
    mask = ~np.isnan(array)
    values = array[mask].ravel()
    classifier = NaturalBreaks(values, k=n_classes)
    return classifier.bins

# Classify data based on bins
def classify_array(array, bins):
    return np.digitize(array, bins, right=True)

def process():
    file1, file2 = select_tiff_files()

    # Confirm file order with OK/Cancel dialog
    root = Tk()
    root.withdraw()
    confirm = messagebox.askokcancel(
        "Confirm File Order",
        f"First selected file (used for HIGH class):\n{os.path.basename(file1)}\n"
        f"Second selected file (used for LOW class):\n{os.path.basename(file2)}\n\n"
        "Click OK to continue or Cancel to stop."
    )
    if not confirm:
        print("Cancelled by user.")
        return

    # Ask for number of classes
    n_classes = simpledialog.askinteger("Input", "Enter number of classes:", initialvalue=5, minvalue=2)
    if not n_classes:
        print("Number of classes not provided.")
        return
        
    # Ask for threshold for second image
    threshold_value = simpledialog.askfloat("Threshold Input", "Enter threshold value for second image (low):")
    if threshold_value is None:
        print("Threshold not provided.")
        return
        
    # Ask for output filename
    output_name = simpledialog.askstring("Output Filename", "Enter output filename (without extension):")
    if not output_name:
        print("Output filename not provided.")
        return

    output_path = os.path.join(os.path.dirname(file1), f"{output_name}.tif")

    # Read raster data
    with rasterio.open(file1) as src1, rasterio.open(file2) as src2:
        arr1 = src1.read(1)
        arr2 = src2.read(1)
        meta = src1.meta.copy()

    # Classify with Natural Breaks
    bins1 = get_class_bins(arr1, n_classes)

    print("\n=== Natural Breaks Thresholds ===")
    print(f"{os.path.basename(file1)} (High class source):")
    for i, b in enumerate(bins1, 1):
        print(f"  Class {i}: <= {b:.4f}")


    class1 = classify_array(arr1, bins1)
    #class2 = classify_array(arr2, bins2)

    # Create mask: high in file1 and low in file2
    high_class = n_classes - 1
    mask = (class1 == high_class) & (arr2 <= threshold_value)

    # Create result
    result = np.zeros_like(arr1, dtype=np.uint8)
    result[mask] = 1

    # Save new raster
    meta.update(dtype='uint8', count=1)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(result, 1)

    print(f"✅ Saved result to: {output_path}")

if __name__ == "__main__":
    process()


=== Natural Breaks Thresholds ===
IRON_probability(XGBoost).tif (High class source):
  Class 1: <= 0.0494
  Class 2: <= 0.1584
  Class 3: <= 0.3496
  Class 4: <= 0.6510
  Class 5: <= 0.9888
✅ Saved result to: D:/Project Rajasthan/Test/XGBOOST\iron_com.tif
